
Cameron Jester
UML HW 2 Part 1


Using Euclidian distance or dot product similarity (choose one per dataset, you can try other similarity metrics).
A) run KMeans on the MNIST Dataset, try K=10
B) run KMeans on the FASHION Dataset, try K=10
C) run KMeans on the 20NG Dataset, try K=20
You can use a library for distance/similarity but you have to implement your own kmeans (EM steps, termination criteria etc).
For all three datasets, evaluate the KMeans objective for a higher K (for example double) or smaller K(for example half).
For all three datasets, evaluate external clustering performance using data labels and performance metrics Purity and Gini Index (see [A] book section 6.9.2).

In [21]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import uniform
from sklearn.metrics.pairwise import euclidean_distances

In [23]:
import torch
import torchvision
from torchvision import datasets

root_dir = '/tmp/data'
#comes as both
train_data = datasets.MNIST(root=root_dir, train=True, download=True)
test_data = datasets.MNIST(root=root_dir, train=False, download=True)


images = torch.cat((train_data.data, test_data.data), dim=0)
labels = torch.cat((train_data.targets, test_data.targets), dim=0)


images_flat = images.view(images.size(0), -1).numpy()


mnist_df = pd.DataFrame(images_flat)
mnist_df['Label'] = labels

normalized_df = mnist_df.drop(columns = 'Label') / 255.0
normalized_df['Label'] = mnist_df['Label']




In [26]:
#https://towardsdatascience.com/create-your-own-k-means-clustering-algorithm-in-python-d7d4c9077670
#used the above website as a starting point

import numpy as np
from sklearn.metrics import pairwise_distances
#originally had a kmeans that wouldnt work after k > 10, cause of list comprehemsion etc
class OptimizedKMeans:
    def __init__(self, k=10, tol=1e-6, verbose=True):
        self.k = k
        self.tol = tol
        self.centroids = None
        self.verbose = verbose

    def initial_centroids(self, X):

        index = np.random.choice(X.shape[0], self.k, replace=False)#picking random centroid within the data
        self.centroids = X[index]

    def distance(self, X):
        return pairwise_distances(X, self.centroids, metric="euclidean") #originally calculating euclidean from formula slowed down computation a lot

    def assign_clusters(self, distances):
        return np.argmin(distances, axis=1) #finding the closest center

    def update_centroids(self, X, labels):
        #m-step here
        new_centroids = np.zeros_like(self.centroids)
        for k in range(self.k):
            cluster_points = X[labels == k]
            if cluster_points.shape[0] > 0:
                new_centroids[k] = np.mean(cluster_points, axis=0)
        return new_centroids

    def compute_objective(self, distances, labels):
        return np.sum([distances[i, labels[i]] ** 2 for i in range(len(labels))])

    def fit(self, X, max_iters=100):

        X = np.array(X)
        self.initial_centroids(X)

        for i in range(max_iters):
            distances = self.distance(X)
            labels = self.assign_clusters(distances)
            new_centroids = self.update_centroids(X, labels)
            objective_value = self.compute_objective(distances, labels)

            centroid_shift = np.linalg.norm(new_centroids - self.centroids)
            if self.verbose:
                print(f"Objective: {objective_value}")

            if centroid_shift < self.tol:
                break

            self.centroids = new_centroids

        return labels

    def predict(self, X):
        distances = self.distance(X)
        return self.assign_clusters(distances)

    def purity_score(self, y_true, y_pred):
        clusters = np.unique(y_pred)
        majority_sum = 0
        for cluster in clusters:
            indices = np.where(y_pred == cluster)
            majority_class = np.bincount(y_true[indices]).argmax()
            majority_sum += np.sum(y_true[indices] == majority_class)
        return majority_sum / len(y_true)

    def gini_index(self, y_true, y_pred):
        clusters = np.unique(y_pred)
        gini = 0
        for cluster in clusters:
            cluster_mask = (y_pred == cluster) #count of when model is right
            cluster_labels = y_true[cluster_mask] #count
            cluster_size = np.sum(cluster_mask)
            gini_cluster = 1 - np.sum((np.unique(cluster_labels, return_counts=True)[1] / cluster_size) ** 2)
            gini += (cluster_size / len(y_true)) * gini_cluster
        return gini




In [28]:
X = normalized_df.values


y_true = normalized_df['Label'].values

kmeans = OptimizedKMeans(k=5)
labels = kmeans.fit(X)
purity = kmeans.purity_score(y_true, labels)
print("Purity Score:", purity)
gini = kmeans.gini_index(y_true, labels)
print("Gini Index:", gini)

Objective: 6151717.157677817
Objective: 3445222.267738994
Objective: 3340910.802118913
Objective: 3302526.6987042204
Objective: 3290293.7116342094
Objective: 3287734.4031196074
Objective: 3286899.788818795
Objective: 3286165.7941167546
Objective: 3285160.8934735684
Objective: 3283629.313437463
Objective: 3281707.367070368
Objective: 3279805.6795046087
Objective: 3278273.7994278893
Objective: 3277156.4055115147
Objective: 3276279.074083939
Objective: 3275623.6920571644
Objective: 3275153.729279093
Objective: 3274876.9712137324
Objective: 3274670.3324048645
Objective: 3274420.708628334
Objective: 3274184.026163076
Objective: 3273944.700541258
Objective: 3273662.668043029
Objective: 3273340.603826941
Objective: 3272976.3680445286
Objective: 3272582.52116741
Objective: 3272174.063435236
Objective: 3271851.3833653764
Objective: 3271624.897773395
Objective: 3271471.351966486
Objective: 3271359.522006662
Objective: 3271283.7890420044
Objective: 3271235.0956199765
Objective: 3271194.328356907


Purity Score kmeans=10 for the MNIST: 0.7827857142857143

Gini Index kmeans=10 for the MNIST:  0.31031970933808695





Purity Score kmeans=20 for the MNIST:0.8757714285714285

Gini Index kmeans=20 for the MNIST: 0.1886065272293594



Purity Score kmeans=5 for the MNIST: 0.5019714285714286

Gini Index kmeans=5 for the MNIST: 0.49164285714285716

In [47]:
filepath = '/content/drive/My Drive/archive-2/fashion-mnist_test.csv'
filepath2 = '/content/drive/My Drive/archive-2/fashion-mnist_train.csv'

train_df = pd.read_csv(filepath2)
test_df = pd.read_csv(filepath)
df = pd.concat((train_df, test_df))

y_true = df['label'].values

df = df.drop(columns=['label'])
df = df/255 #normalizing

X = df.values

kmeans = OptimizedKMeans(k=20)
labels = kmeans.fit(X)


purity = kmeans.purity_score(y_true, labels)
print("Purity Score:", purity)
gini = kmeans.gini_index(y_true, labels)
print("Gini Index:", gini)


Objective: 3461110.176655131
Objective: 2084923.28291133
Objective: 2035417.4575600813
Objective: 2010338.477291605
Objective: 1994593.1186896223
Objective: 1982362.9583775424
Objective: 1969719.6109198555
Objective: 1951534.32630745
Objective: 1927756.0926431692
Objective: 1911374.988933131
Objective: 1905010.6649034987
Objective: 1901967.6100282343
Objective: 1899927.7587298215
Objective: 1898351.0147084412
Objective: 1897268.5010500744
Objective: 1896466.6410961035
Objective: 1895856.1530222988
Objective: 1895348.9131564037
Objective: 1894940.6550407554
Objective: 1894583.0346824068
Objective: 1894191.3378963822
Objective: 1893732.9554810245
Objective: 1893274.2733797904
Objective: 1892704.4159230355
Objective: 1892129.5773013122
Objective: 1891469.6260367495
Objective: 1890749.0243982517
Objective: 1890015.1539525934
Objective: 1889163.619251289
Objective: 1888222.9909900525
Objective: 1887188.4369507916
Objective: 1886443.3973584273
Objective: 1885983.6259224534
Objective: 1885745

Purity Score kmeans=10 for the FASHION MNIST: 0.5546428571428571

Gini Index kmeans=10 for the FASHION MNIST: 0.5399887747713308



Purity Score kmeans= 20 for the FASHION  MNIST: 0.6545857142857143


Gini Index kmeans= 20 for the FASHION MNIST: 0.4511777137250461



Purity Score kmeans=5 for the FASHION  MNIST: 0.4107142857142857

Gini Index kmeans=5 for the FASHION MNIST: 0.7070143912323185



In [2]:
#JUST USING SAME DATA PREP FROM HW1

import numpy as np
from sklearn.metrics.pairwise import cosine_distances
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pandas as pd


newsgroup_data = fetch_20newsgroups(subset="test")
texts = newsgroup_data.data
newsgroup = newsgroup_data.target

vectorizer = CountVectorizer(stop_words='english')
tfidf = TfidfTransformer()


text_vectorized = vectorizer.fit_transform(texts)
text_tf = tfidf.fit_transform(text_vectorized)


ng_df = pd.DataFrame(text_tf.toarray())
ng_df['Label'] = newsgroup


In [10]:
import numpy as np
from sklearn.metrics import pairwise_distances

class Kmeans20NG:
  def __init__(self, k=10, max_iters = 100,  tol=1e-6, verbose=True):
        self.k = k
        self.max_iters = max_iters
        self.tol = tol
        self.centroids = None
        self.verbose = verbose
  def initial_centroids(self, X):
    np.random.seed(510)
    index = np.random.choice(X.shape[0], self.k, replace = False)
    self.centroids = X[index]

  def distance(self, X):
    return 1 - pairwise_distances(X, self.centroids, metric='cosine')

  def assign_clusters(self, distances):
    return np.argmax(distances, axis=1)

  def update_centroids(self, X, labels):
    updated_centroids = np.zeros_like(self.centroids)
    for k in range(self.k):
      cluster_points = X[labels ==k]
      if cluster_points.shape[0] > 0:
        updated_centroids[k] = np.mean(cluster_points, axis=0)
    return updated_centroids

  def compute_objective(self, distances, labels):
    return np.sum([distances[i, labels[i]] ** 2 for i in range(len(labels))])

  def fit(self, X):
    self.initial_centroids(X)

    for i in range(self.max_iters):
      distances = self.distance(X)
      labels = self.assign_clusters(distances)
      new_centroids = self.update_centroids(X, labels)
      objective_value = self.compute_objective(distances, labels)

      centroid_shift = np.linalg.norm(new_centroids - self.centroids)
      if self.verbose:
        print(f"Objective: {objective_value}")
      if centroid_shift < self.tol:
        break

      self.centroids = new_centroids

    return labels

  def predict(self, X):
    distances = self.distance(X)
    return self.assign_clusters(distances)

  def purity_score(self, y_true, y_pred):
    clusters = np.unique(y_pred)
    majority_sum = 0
    for cluster in clusters:
      index = np.where(y_pred == cluster)
      majority_class = np.bincount(y_true[index]).argmax()
      majority_sum += np.sum(y_true[index] == majority_class)
    return majority_sum / len(y_true)

  def gini_index(self, y_true, y_pred):
        clusters = np.unique(y_pred)
        gini = 0
        for cluster in clusters:
            cluster_mask = (y_pred == cluster) #count of when model is right
            cluster_labels = y_true[cluster_mask] #count
            cluster_size = np.sum(cluster_mask)
            gini_cluster = 1 - np.sum((np.unique(cluster_labels, return_counts=True)[1] / cluster_size) ** 2)
            gini += (cluster_size / len(y_true)) * gini_cluster
        return gini



kmeans = Kmeans20NG(k = 20, max_iters= 100, tol = 1e-6)

X = ng_df.values
y_true = ng_df['Label'].values


y_pred = kmeans.fit(X)

distances = kmeans.distance(X)
objective_value = kmeans.compute_objective(distances, y_pred)
print("Objective Value:", objective_value)
purity = kmeans.purity_score(y_true, y_pred)
print("Purity Score:", purity)
gini = kmeans.gini_index(y_true, y_pred)
print("Gini Index:", gini)







Objective: 6797.274744146243
Objective: 6833.413374063614
Objective: 6848.5592393616225
Objective: 6852.239454760565
Objective: 6856.169338172969
Objective: 6857.719279831303
Objective: 6858.05481186882
Objective: 6858.189868968489
Objective: 6858.478967559733
Objective: 6858.543695288893
Objective: 6858.647693544663
Objective: 6858.683084505739
Objective: 6858.69787129888
Objective: 6858.69907261757
Objective Value: 6858.69907261757
Purity Score: 0.21109930961232076
Gini Index: 0.8136400280804393


Purity Score kmeans=10 for the 20NG: 0.17219861922464152


Gini Index kmeans=10 for the 20NG: 0.477318085316


Purity Score kmeans=20 for the 20NG:  0.21109930961232076


Gini Index kmeans=20 for the 20NG: 0.8136400280804393

Purity Score kmeans=5 for the 20NG: 0.1541423260754116

Gini Index kmeans=5 for the 20NG: 0.63642321976






Now I am doing the soft k means:

In [5]:
class softKMeans():
    def __init__(self, k=10, tol=1e-6, beta = 0.1):
        self.k = k
        self.tol = tol
        self.beta = beta

    def fit(self, df, max_iter = 100):
        df = np.array(df)  # easy enough, turning to an array
        min_, max_ = np.min(df, axis=0), np.max(df, axis=0)  # finding the min and max across columns
        self.centroids = [np.random.uniform(min_, max_) for index in range(self.k)]  #I think this is still good to add for soft k means, since we still need centroids
        # above it is randomly placing centroids for the number of clusters

        iteration = 0
        prev_centroids = None  # just initializing the centroids

        for iteration in range(max_iter):  #looping until tolerence is hit
            dists = np.linalg.norm(df[:, None, :] - self.centroids, axis=2)


            responsibilities = np.exp(-self.beta * dists)  #here is where we are calculating
            responsibilities /= np.sum(responsibilities, axis=1, keepdims=True)  #normalizing


            #sorted_points = [[] for _ in range(self.k)]  # starts off with empty list for each centroid
            #dists = np.linalg.norm(df[:, None, :] - np.array(self.centroids), axis=2)  # need to figure out what this is
            #cluster_indices = np.argmin(dists, axis=1)  # assigning each point to the closest cluster
            #sorted_points = [df[cluster_indices == i] for i in range(self.k)]  # sorting points into clusters based on the closest centroid

            prev_centroids = self.centroids.copy()
            self.centroids = (responsibilities .T @ df) / np.sum(responsibilities, axis=0)[:, None]  #just getting weighted average


            for i, centroid in enumerate(self.centroids):
                if np.isnan(centroid).any():
                    self.centroids[i] = prev_centroids[i]

            centroid_diff = np.sum((np.array(self.centroids) - np.array(prev_centroids)) ** 2)


            if centroid_diff < self.tol:
                print(f"Converged after {iteration} iterations.")
                break

    def euclidean(self, point, data):
        return np.sqrt(np.sum((point - data) ** 2, axis=1))

    def evaluate(self, X):
        X = np.array(X)
        dists = np.linalg.norm(X[:, None, :] - self.centroids, axis=2)
        responsibilities = np.exp(-self.beta * dists)
        responsibilities /= np.sum(responsibilities, axis=1, keepdims=True)
        return responsibilities


In [16]:
X = normalized_df.values

softkmeans = softKMeans(beta = 1)
softkmeans.fit(X)

In [9]:
def purity_score(y_true, y_pred):
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

from sklearn import metrics

In [12]:
y_pred.shape

(70000, 10)

In [13]:
normalized_df['Label'].values.shape

(70000,)

In [17]:
y_pred = np.argmax(softkmeans.evaluate(X), axis=1)

y_true = normalized_df['Label'].values
purity = purity_score(y_true, y_pred)
print("Purity Score for softKmeans:", purity)

Purity Score for softKmeans: 0.4432142857142857


Purity Score for softKmeans beta = 0.1: 0.21748571428571428


Purity Score for softKmeans beta = 1:  0.4432142857142857


Purity Score for softKmeans beta = 10: 0.64956432611